- crear programa que cumpla con las normas
- ser open source, libre y multiplataforma
- encuentrar una seccion optima
- replicar caracteristicas de otros programas
- calidad y no novedad
- apoyo para la docencia
- ramos aplicados al programa
- difusion de informacion
- conflicto de patentes (precios y licencias)
- soluciones digitales
- problemas del cracking
- pirateria en clases
- apoyo y reconocimiento al software libre
- carrera de programador
- caso de chile vs latinoamerica


In [1]:
from numpy import array, zeros, ones
from numpy.linalg import inv, solve
def line(): print('---- -- - -- ----')

## Input de Usuario

Para poder ingresar los datos al programa, se espera que el usuario
ingrese en un orden establecido la siguiente informacion para la
correcta definicion de los marcos a analizar.

123456789012345678901234567890123456789012345678901234567890123456789

```
numero de pisos          : 1
numero habitaciones      :1
numero de marcos         :1
altura de piso           :300
distancia entre columnas :200
distancia entre marcos   :0

modulo de poisson 𝜈      :0.25
resistencia del hormigon :19.5
densidad del hormigon    : 2500
carga viva               :0

zona sismica             :
tipo de suelo            :
categoria de edificacion :

resistencia del acero    :
tama;o max de columnas   :
altura max de viga       :
```

In [2]:
# recuperacion de datos

with open('../IO/input.txt', encoding='utf-8') as i_text:
    i_temp = i_text.readlines()

default = {
    'numero de pisos': '1',
    'numero habitaciones': '1',
    'numero de marcos': '1',
    'altura de piso': '300',
    'distancia entre columnas': '200',
    'distancia entre marcos': '0',
    'modulo de poisson 𝜈': '0.25',
    'resistencia del hormigon': '19.5',
    'densidad del hormigon': '2500',
    'carga viva': '0',
    'zona sismica': 'def',
    'tipo de suelo': 'def',
    'categoria de edificacion': 'def',
    'resistencia del acero': 'def',
    'tama;o max de columnas': 'def',
    'altura max de viga': 'def',
}

user_op = []
user_in = []

for i in i_temp:
    if i.startswith('\n') == False:
        user_op.append(i.partition(':')[0].strip())
        user_in.append(i.partition(':')[2].strip())
print(user_in)
del i_temp
if user_op != list(default.keys()): raise Exception('wena')

for i in range(len(user_in)):
    if user_in[i] == '':
        user_in[i] = list(default.values())[i]
    elif user_in[i].isalpha() == False:
        user_in[i] = float(user_in[i])
for i in range(3): user_in[i] = int(user_in[i])
print(user_in)

['1', '1', '1', '300', '200', '0', '0.25', '19.5', '2500', '0', '', '', '', '', '', '']
[1, 1, 1, 300.0, 200.0, 0.0, 0.25, 19.5, 2500.0, 0.0, 'def', 'def', 'def', 'def', 'def', 'def']


## Grilla

Para definir el espacio donde sera colocado cada elemento, se define
una grilla multidimencional donde se reflejaran los datos de inicio
y fin de cada elemento (viga o columna), la interaccion entre ellos y
los apoyos.

123456789012345678901234567890123456789012345678901234567890123456789

In [3]:
class Grilla():
    def __init__(grid, roofs, rooms, frames, height, width, depth):
        grid.roofs = roofs
        grid.rooms = rooms
        grid.frames = frames
        grid.nodes = [i for i in range(1, roofs*(rooms + 1)*frames + 1)]
        grid.foots = (rooms + 1)*frames
        print(grid.nodes, grid.foots) # del

In [4]:
def set_frames(roofs, rooms, frames, height, width, depth):
    MARCOS = (
        Grilla(roofs, rooms, frames, height, width, depth),
        Grilla(roofs, rooms, 1, height, width, depth),
        Grilla(roofs, frames - 1, 1, height, width, depth),
        Grilla(1, rooms, frames, height, width, depth)
    )
    return MARCOS

In [5]:
malla = Grilla(2, 3, 2, 0, 0, 0)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16] 8


In [6]:
mallaXY = Grilla(2, 3, 1, 0, 0, 0)

[1, 2, 3, 4, 5, 6, 7, 8] 4


In [7]:
mallaZY = Grilla(2, 1, 1, 0, 0, 0)

[1, 2, 3, 4] 2


In [8]:
mallaZX = Grilla(1,3,2, 0, 0, 0)

[1, 2, 3, 4, 5, 6, 7, 8] 8


In [9]:
set_frames(2, 1, 4, 0, 0, 0);

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16] 8
[1, 2, 3, 4] 2
[1, 2, 3, 4, 5, 6, 7, 8] 4
[1, 2, 3, 4, 5, 6, 7, 8] 8


## Cargas nodales

A partir de un analisis de una viga doblemente empotrada, se calculan
las reacciones de esta para cada caso, considerando la continuidad de
una losa que descargue sobre el marco. Para esto se consideran las
formulas de empotramiento perfecto de cargas lineales y continuas. De
este modo, se obtiene la carga trapezoidal resultante de la
superposicion de tres casos de cargas por viga.

123456789012345678901234567890123456789012345678901234567890123456789

### Formula Carga Constante

In [10]:
def q_rec(L, q, a, b, d):
    Ra = q*d/L**3*((2*a + L)*b**2 + (a - b)/4*d**2)
    Rb = q*d/L**3*((2*b + L)*a**2 - (a - b)/4*d**2)
    Ma = q*d/L**2*(a*b**2 + d**2/12*(L - 3*b))
    Mb = -q*d/L**2*(a**2*b + d**2/12*(L - 3*a))

    print(round(Ra, 1), round(Rb, 1), round(Ma, 1), round(Mb, 1))
    return Ra, Rb, Ma, Mb

In [11]:
l = 2
q = 1000
d = 1.2
a = 0.5 + d/2
b = l - a

q_rec(l, q, a, b, d);

521.1 678.9 242.1 -279.9


### Formula Carga Lineal

In [12]:
def q_tri(L, q, a):
    Ra = q*a/20*(10 - a**2/L**2*(15 - 8*a/L))
    Rb = q*a**3/(20*L**2)*(15-8*a/L)
    Ma = q*a**2/30*(10 - a/L*(15 - 6*a/L))
    Mb = -q*a**3/(20*L)*(5 - 4*a/L)
    print(round(Ra, 1), round(Rb, 1), round(Ma, 1), round(Mb, 1))
    return Ra, Rb, Ma, Mb

In [13]:
l = 2
q = 1000
a = 0.5
b = l - a

q_tri(l, q, a);

229.7 20.3 55.2 -12.5


In [14]:
# superposicion de casos de carga de una viga cualquiera

r = [[],[],[]]
joints = []

L = 2
a = 0.5
b = 0.3
d = 1.2
q = 1000

*r[0], = q_tri(L, q, a)
*r[1], = q_rec(L, q, a + d/2, b + d/2, d)
*r[2], = q_tri(L, q, b)

r[2][0], r[2][1] = r[2][1], r[2][0]
r[2][2], r[2][3] = -r[2][3], -r[2][2]

for i in range(4):
    joints.append(0)
    for j in range(3):
        joints[i] += r[j][i]

line()

print('Va:', round(joints[0], 1))
print('Vb:', round(joints[1], 1))
print('Ma:', round(joints[2], 1))
print('Mb:', round(joints[3], 1))

229.7 20.3 55.2 -12.5
521.1 678.9 242.1 -279.9
145.3 4.7 23.7 -3.0
---- -- - -- ----
Va: 755.4
Vb: 844.6
Ma: 300.3
Mb: -316.1


## Definicion de Elementos

A continuacion se define cada uno de los parametros y sus respectivas
funciones para un objeto _Barra_ generico.

123456789012345678901234567890123456789012345678901234567890123456789

In [15]:
class barra():
    # user input at init
    # i j
    # x0 y0 x1 y1 # [cm]
    # b = 30 # [cm]
    # h = 40 # [cm]
    # ϰ = 1.2 # factor de forma de la seccion
    # 𝜈 = 0.25 # modulo de poisson
    # fc = 19.5 # [MPa]
    #
    # internals
    # L = ((x1 - x0)**2 + (y1 - y0)**2)**0.5 # [cm]
    # s = (y1 - y0)/L # seno
    # c = (x1 - x0)/L # coseno
    # A = b*h # [cm2]
    # I = b*h**3/12 # [cm4]
    # E = (15100*(fc*1000)**0.5/100)/0.01 # [kg/mm2]/0.01 = [kg/cm2]
    # G = E/(2*(1+𝜈)) # modulo de cizalle
    # β = 6*E*I*ϰ/(G*A*L**2)
    # k11 = 2*E*I*(2 + β)/(L*(1 + 2*β))
    # k12 = 2*E*I*(1 - β)/(L*(1 + 2*β))
    # k21 = k12
    # k22 = k11
    # k33 = A*E/L
    # k = array([
    #     [k11, k12, 0],
    #     [k21, k22, 0],
    #     [0,   0,   k33]
    # ]) # rigidez local
    # a = array([
    #     [-s/L, c/L, 1,  s/L, -c/L, 0],
    #     [-s/L, c/L, 0,  s/L, -c/L, 1],
    #     [c,    s,   0, -c,   -s,   0]
    # ]) # compatibilidad geometrica
    # K = a.T@k@a
    # σ = k@a@T@r_global

    def __init__(bar, nodos, i, j, x0, y0, x1, y1, b, h, ϰ, 𝜈, fc):
        #### ## input ## ####
        bar.i = i # nodo A
        bar.j = j # nodo B
        bar.x0 = x0 # [cm]
        bar.y0 = y0 # [cm]
        bar.x1 = x1 # [cm]
        bar.y1 = y1 # [cm]
        bar.b = b # [cm]
        bar.h = h # [cm]
        bar.ϰ = ϰ # factor de forma de la seccion
        bar.𝜈 = 𝜈 # modulo de poisson
        bar.fc = fc # [MPa]

        #### ## internals ## ####
        bar.GdL(i, j, nodos)

        bar.L = ((x1 - x0)**2 + (y1 - y0)**2)**0.5 # [cm]
        bar.s = (y1 - y0)/bar.L # seno
        bar.c = (x1 - x0)/bar.L # coseno
        bar.A = b*h # [cm2]
        bar.I = b*h**3/12 # [cm4]
        bar.E = (15100*(fc*1000)**0.5/100)/0.01 # [kg/mm2]/0.01 = [kg/cm2]
        bar.G = bar.E/(2*(1+𝜈)) # modulo de cizalle
        bar.β = 6*bar.E*bar.I*ϰ/(bar.G*bar.A*bar.L**2)

        bar.rigidez_local(bar.L, bar.A, bar.I, bar.E, bar.β)
        bar.compatibilidad_geometrica(bar.s, bar.c, bar.L)

        bar.K = bar.a.T@bar.k@bar.a # rigidez global
        
    def GdL(bar, a, b, nodos, tipo=''):
        bar.T = zeros([6, 3*nodos])
        if a > nodos or b > nodos:
            print('index error')
            return
        if a < 0 or b < 0:
            print('index error')
            return
        if a != 0:
            bar.T[0, 3*a - 3] = 1
            bar.T[1, 3*a - 2] = 1
            bar.T[2, 3*a - 1] = 1
        if b != 0:
            bar.T[3, 3*b - 3] = 1
            bar.T[4, 3*b - 2] = 1
            bar.T[5, 3*b - 1] = 1

    def compatibilidad_geometrica(bar, s, c, L):
        bar.a = array([
            [-s/L, c/L, 1,  s/L, -c/L, 0],
            [-s/L, c/L, 0,  s/L, -c/L, 1],
            [ c,   s,   0, -c,   -s,   0]
        ])

    def rigidez_local(bar, L, A, I, E, β):
        k11 = 2*E*I*(2 + β)/(L*(1 + 2*β))
        k12 = 2*E*I*(1 - β)/(L*(1 + 2*β))
        k21 = k12
        k22 = k11
        k33 = A*E/L
        bar.k = array([
            [k11, k12, 0  ],
            [k21, k22, 0  ],
            [0,   0,   k33]
        ])

    def sigma(bar, r_global):
        # σ = k@ε
        # ε = a@r_i
        # r_i = T@r
        # σ = k@a@T@r
        bar.σ = bar.k@bar.a@bar.T@r_global # esfuerzos

    r = 0 # desplazamientos - sin uso
    ε = 0 # deformaciones - sin uso
    σ = 0 # esfuerzos

In [16]:
#__init__(nodos, i, j,  x0, y0, x1,  y1,  b,  h, ϰ,    𝜈,   fc)
col = barra(  5, 2, 5, 200, 0, 200, 300, 30, 40, 0, 0.25, 19.5)
print(col.T.T@ones((6, 6))@col.T)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1.]]
